In [46]:
with open("cpp/res.txt", "r") as file:
    content = file.read()

In [47]:
xhs=eval(content)

In [23]:
!pip3 install statsmodels
from double_line_single import double_line


Defaulting to user installation because normal site-packages is not writeable


In [48]:
xhs.sort(key=lambda x:x[1])

In [49]:
xh2d=[xhs[i*11:(i+1)*11] for i in range(11) ]

In [50]:
for i in xh2d:
    i.sort(key=lambda x:x[0])

In [39]:
! pip3 install opencv-python-headless
!pip3 install matplotlib 
import cv2
import matplotlib.pyplot as plt

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 54.1 MB/s eta 0:00:00:00:0100:01
Defaulting to user installation because normal site-packages is not writeable


In [42]:
file='2Y0Y3B1XSDG8Q002H_G.tiff'
# Read the TIFF file with IMREAD_ANYDEPTH flag to handle 32-bit depth
image = cv2.imread(file, cv2.IMREAD_ANYDEPTH)

# Convert to float if necessary (OpenCV reads it as uint16 or similar)
image = image.astype('float32')


In [44]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [53]:
import pyppt as ppt
import matplotlib.pyplot as plt
import numpy as np

def cropNenhance(image):
    # Step 2: Crop the image (20% from each edge)
    height, width = image.shape[:2]
    crop_height = int(height * 0.2)
    crop_width = int(width * 0.2)
    
    # Cropping the image
    cropped_image = image[crop_height:height-crop_height, crop_width:width-crop_width]
    
    # Step 3: Enhance the image to scale pixel values to [0, 255]
    # Normalize the pixel values
    min_val = np.min(cropped_image)
    max_val = np.max(cropped_image)
    
    # Scale to [0, 255]
    enhanced_image = (255 * (cropped_image - min_val) / (max_val - min_val)).astype(np.uint8)
    return enhanced_image



# # Add the figure to the PowerPoint slide
# ppt.add_figure('Center')

import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches

# Create matplotlib figure
# plt.plot([1, 2, 3, 4])

plt.imshow(cropNenhance(image), cmap='gray')
# Adjust the layout
plt.tight_layout()
plt.savefig('graph.png')
plt.close()

# Add to PowerPoint
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[6])
slide.shapes.add_picture('graph.png', Inches(1), Inches(1))
prs.save('pngdd.pptx')

In [66]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

# Function to determine text color based on background luminance
def get_text_color(value, vmin, vmax, cmap):
    norm = plt.Normalize(vmin, vmax)
    rgba = cmap(norm(value))
    luminance = 0.299 * rgba[0] + 0.587 * rgba[1] + 0.114 * rgba[2]
    return 'black' if luminance > 0.5 else 'white'

# Create a custom colormap
def create_custom_ylord():
    # Get the YlOrRd colormap
    ylord = plt.cm.YlOrRd(np.linspace(0, 1, 256))
    
    # Create a new colormap with white at the beginning
    custom_ylord = np.vstack((np.array([1, 1, 1, 1]), ylord))
    return colors.ListedColormap(custom_ylord)

fig, ax = plt.subplots(figsize=(7,7))
ax.set_aspect('equal') 
y=x=[i for i in range(1,12)]
# Create a modified YlOrRd colormap with white as the minimum
cmap = plt.cm.YlOrRd.copy()
cmap.set_under('white')
    
# Create the custom colormap
cmap = create_custom_ylord()

heatmap = ax.pcolormesh(x, y, res_map, shading='auto', cmap=cmap,edgecolors='grey',
                       linewidths=0.5)

# Add a colorbar
# plt.colorbar(heatmap)
# # Use imshow with origin='upper'
# im = ax.imshow(res_map, origin='upper')

# Add value annotations
max_value = max(max(row) for row in res_map)
min_value = min(min(row) for row in res_map)
for i in range(len(y)):
    for j in range(len(x)):
        ax.text(x[j], y[i], f'{res_map[i][j]}', 
                ha='center', va='center', color=get_text_color(res_map[i][j],
                                                              min_value,max_value ,cmap))
        
# Set labels and title
# ax.set_xlabel('Columns')
# ax.set_ylabel('Rows')
# Adjust axes
# ax.set_xlim(0,10)
# Get the current y-axis limits
y_min, y_max = ax.get_ylim()
ax.set_ylim(y_max,y_min) 
ax.set_xticks(x)
ax.set_xticklabels([f'C{i}' for i in x]) 
ax.set_yticks(y)
ax.set_yticklabels([f'R{i}' for i in y]) 
# ax.set_title('2D Heatmap')
# Move x-axis to the top
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')

plt.savefig('graph2.png')
plt.close()

In [54]:
def cropNenhance(image,left,right,top,bottom):
    # Step 2: Crop the image (20% from each edge)
#     height, width = image.shape[:2]
#     crop_height = int(height * 0.2)
#     crop_width = int(width * 0.2)
    
    # Cropping the image
#     cropped_image = image[crop_height:height-crop_height, crop_width:width-crop_width]
    cropped_image=image[top:bottom+1,left:right+1]
    
    # Step 3: Enhance the image to scale pixel values to [0, 255]
    # Normalize the pixel values
    min_val = np.min(cropped_image)
    max_val = np.max(cropped_image)
    
    # Scale to [0, 255]
    enhanced_image = (255 * (cropped_image - min_val) / (max_val - min_val)).astype(np.uint8)
    return enhanced_image

res_map=[[0]*11 for i in range(11)]
for x in range(11):
    for y in range(11):
        [x0,y0]=xh2d[y][x]
        res=double_line(cropNenhance(image,x0-80,x0+80,y0-80,y0+80))
        res_map[y][x]=res['lvl']

In [69]:
# Create matplotlib figure
# plt.plot([1, 2, 3, 4])
filename='2Y0Y3B1XSDG8Q002H_G.tiff'
height, width = image.shape[:2]
gridleft=xh2d[0][0][0]
gridtop=xh2d[0][0][1]
gridright=xh2d[10][10][0]
gridbottom=xh2d[10][10][1]
print(gridtop, gridbottom)

cropleft=int(max(gridleft-(gridright-gridleft)/10,0))
cropright=int(min(gridright+(gridright-gridleft)/10,int(width)))

croptop=int(max(gridtop-(gridbottom-gridtop)/10,0))
cropbottom=int(min(gridbottom+(gridbottom-gridtop)/10,int(height)))

plt.figure(figsize=(7, 7))
plt.imshow(cropNenhance(image,cropleft, cropright, croptop, cropbottom), cmap='gray')
plt.title(filename.split('/')[-1])
# Adjust the layout
plt.tight_layout()
plt.savefig('graph.png')
plt.close()

# Add to PowerPoint
prs = Presentation()
# Set slide dimensions to 16:9 aspect ratio
prs.slide_width = Inches(16)
prs.slide_height = Inches(9)

slide = prs.slides.add_slide(prs.slide_layouts[6])
slide.shapes.add_picture('graph.png', Inches(0.5), Inches(1))

slide.shapes.add_picture('graph2.png', Inches(8), Inches(1))

prs.save('pngdd.pptx')

783 3523
